In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch import nn 
import torch.optim as optim
from torch.distributions import Normal


# Teaching a quadruped to walk

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible.

Your goal is to implement a class `WalkerPolicy` with function `determine_actions()` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the `determine_actions()` function!

After you implement it, copy `WalkerPolicy` into a separate file `WalkerPolicy.py` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.


# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1. This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2. You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3. Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable `torch.Parameter` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4. The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term.

5. If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key `'reward_fcn'`. See the `WalkerEnv` class for the implementation of the default reward.

6. Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the `torch.distributions.beta` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)


In [2]:
# If you cannot run with the visualization, you can set this to False
VISUALIZE = True

In [3]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy

In [4]:
def sample_trajectories(env, policy, T=100):
    states_np = env.vector_reset() 
    states = [states_np]
    actions = []
    rewards = []

    for t in range(T):
        s_torch = torch.tensor(states_np, dtype=torch.float32)
        a_torch = policy.sample_actions(s_torch)       
        a_np = a_torch.detach().cpu().numpy()

        next_states_np, r_np = env.vector_step(a_np)  
        actions.append(a_np)
        rewards.append(r_np)
        states.append(next_states_np)
        states_np = next_states_np

    states_t = torch.tensor(np.array(states), dtype=torch.float32)
    actions_t = torch.tensor(np.array(actions), dtype=torch.float32) 
    rewards_t = torch.tensor(np.array(rewards), dtype=torch.float32)    
    return states_t, actions_t, rewards_t

In [5]:
def compute_returns(rewards, gamma=0.99):
    T, N = rewards.shape
    returns = torch.zeros_like(rewards)
    running_sum = torch.zeros(N)
    for t in reversed(range(T)):
        running_sum = rewards[t] + gamma * running_sum
        returns[t] = running_sum
    return returns


def compute_returns_and_advantages(
    rewards, values, dones, gamma=0.99, lam=0.95
):
    T, N = rewards.shape
    returns = np.zeros((T, N), dtype=np.float32)
    advantages = np.zeros((T, N), dtype=np.float32)

    gae = 0.0
    for t in reversed(range(T)):
        mask = 1.0 - dones[t]
        delta = rewards[t] + gamma * values[t + 1] * mask - values[t]
        gae = delta + gamma * lam * mask * gae
        advantages[t] = gae
        returns[t] = advantages[t] + values[t]

    return returns, advantages



In [7]:
def train_walker(env, policy, epochs=100, T=1000, gamma=0.99, lr=1e-3):
    optimizer = optim.Adam(policy.parameters(), lr=lr)

    for e in range(epochs):
        states, actions, rewards = sample_trajectories(env, policy, T=T)
        T_, N_ = rewards.shape
        
        # values = policy.value_estimates(states)
        G_t = compute_returns(rewards, gamma=gamma) 
        
        flat_states = states[:T].reshape(T_ * N_, -1)
        flat_actions = actions.reshape(T_ * N_, -1)
        flat_returns = G_t.reshape(T_ * N_)

        logp = policy.log_prob(flat_states, flat_actions)  
        loss = -(logp * flat_returns).mean()
        
        # with torch.no_grad():

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_reward = rewards.mean().item()
        print(f"Epoch {e + 1}/{epochs}, Loss={loss.item():.3f}, AvgReward={avg_reward:.3f}")

    print("Training done!")

In [8]:
def test_walker(env, policy, T=1000):
    obs = env.vector_reset() 
    N = env.num_walkers
    total_reward = 0.0
    max_x = obs[0, 0] if N == 1 else 0.0

    for t in range(T):
        a = policy.determine_actions(torch.tensor(obs).float())
        obs, reward = env.vector_step(a.detach().numpy())
        total_reward += reward.sum()       
        if N == 1:
            max_x = max(max_x, obs[0, 0])

    print(f"Test completed. Total reward = {total_reward:.2f}, Max X = {max_x:.2f}")
    return total_reward, max_x

In [ ]:
config = {
    "N": 16,            
    "vis": False,      
    "track": 0,        
    # "reward_fcn": walker_reward
}
env = WalkerEnv(config)

policy = WalkerPolicy(state_dim=29, action_dim=8)

train_walker(env, policy, epochs=50, T=500, gamma=0.99, lr=1e-3)

Environment ready
Epoch 1/50, Loss=-9.522, AvgReward=-0.033
Epoch 2/50, Loss=-11.209, AvgReward=-0.039
Epoch 3/50, Loss=-7.317, AvgReward=-0.022
Epoch 4/50, Loss=-6.987, AvgReward=-0.023
Epoch 5/50, Loss=-3.507, AvgReward=-0.012
Epoch 6/50, Loss=2.550, AvgReward=0.009
Epoch 7/50, Loss=2.487, AvgReward=0.006
Epoch 8/50, Loss=8.020, AvgReward=0.024
Epoch 9/50, Loss=3.209, AvgReward=0.008
Epoch 10/50, Loss=7.481, AvgReward=0.024
Epoch 11/50, Loss=10.780, AvgReward=0.034


In [10]:
config = {
    "N": 1,           
    "vis": True,     
    "track": 0,        
    # "reward_fcn": walker_reward
}
env = WalkerEnv(config)
test_walker(env, policy, T=500)

env.close()

Environment ready


2025-01-02 15:23:07.444 Python[75865:9347566] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-02 15:23:07.444 Python[75865:9347566] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/Users/hella/.venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


Test completed. Total reward = -0.62, Max X = 0.02
